In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [2]:
#Merged data from the diagnostics.csv and faults.csv
combined = pd.read_csv("../big_g_express-domo-arigato-mr-robot/combined.csv")

/home/evgeniya/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
combined['EventTimeStamp'] = pd.to_datetime(combined['EventTimeStamp'])
combined["EquipmentID"]= combined["EquipmentID"].astype(str)


In [ ]:
#fully_derated_preceding = combined.loc[combined['EquipmentID'].isin(fully_derated['EquipmentID']), :]

In [4]:
combined['fully_derated'] = np.where((combined['spn'] ==5246) & (combined['fmi'] ==0), 1, 0)

In [6]:
combined['partially_derated'] = np.where((combined['spn'] ==1569) & (combined['fmi'] ==31), 1, 0)

In [7]:
combined.head()

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,fully_derated,partially_derated
0,0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,78.8,1023.0,True,NaN,0.0,3276.75,NaN,0.0,0,0
1,1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,...,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,16639.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0


## ADD FIRST DERATE DATE FOR FULLY DERATED TRUCKS PER A TRUCK

In [8]:
combined.shape

(1187335, 50)

In [9]:
combined_min_date = combined.loc[combined['fully_derated'] ==1].groupby('EquipmentID').agg({'EventTimeStamp' : 'min'}).reset_index()

In [10]:
combined_min_date.rename(columns={'EventTimeStamp': 'first_derate_date'}, inplace=True)

In [11]:
combined = combined.sort_values(by = ['EventTimeStamp', 'EquipmentID'], ascending = [True, True])

In [12]:
combined_min_date.sort_values(by = ['first_derate_date', 'EquipmentID'], ascending = [True, True])

,EquipmentID,first_derate_date
200,2175,2011-01-01 00:03:23
118,1751,2011-01-01 00:03:34
29,1452,2011-01-01 00:04:57
14,1395,2015-02-21 12:10:51
92,1630,2015-02-23 05:05:44
...,...,...
177,1947,2019-09-30 04:56:43
196,2089,2019-11-01 18:07:51
202,2211,2020-02-03 16:34:21
150,1854,2020-02-06 07:45:08


In [13]:
combined = combined.merge(combined_min_date, on = ['EquipmentID'], how = 'left')

In [14]:
#indicate when truck id changes
combined['flipper'] = np.where((combined.EquipmentID != combined.EquipmentID.shift(1)), 1, 0)

In [15]:
combined = combined[combined['EquipmentID'].str.len()<6]

In [16]:
combined = combined[~((combined['Longitude'].astype('str').str.contains(r'-86.44\d+')) & (combined['Latitude'].astype('str').str.contains(r'-36.06\d+')))]

In [17]:
combined = combined[~((combined['Longitude'].astype('str').str.contains(r'-86.44\d+')) & (combined['Latitude'].astype('str').str.contains(r'-35.58\d+')))]

In [18]:
combined = combined[~((combined['Longitude'].astype('str').str.contains(r'-83.17\d+')) & (combined['Latitude'].astype('str').str.contains(r'-36.19\d+')))]

In [19]:
combined.shape

(1185166, 52)

In [20]:
#take out recs before 2015
combined = combined.loc[pd.DatetimeIndex(combined['EventTimeStamp']).year >=2015]
combined.shape

(1184648, 52)

## Grab all recodst before first derate signal

In [26]:
before_first_derate = combined.loc[combined['EventTimeStamp'] <=combined['first_derate_date']]

In [33]:
bfd = before_first_derate[(before_first_derate['fully_derated']==1) | (before_first_derate['partially_derated']==1)]

In [49]:
bfd_preceding = combined.loc[combined['EquipmentID'].isin(bfd['EquipmentID']), :]

In [36]:
#this was for partially and fully derated. The res was 350K records and 274 unique spns
#bfd_preceding = combined.loc[combined['EquipmentID'].isin(bfd['EquipmentID']), :]

In [50]:
bfd_preceding.set_index(['EventTimeStamp','EquipmentID'],inplace=True)

In [51]:
bfd_preceding_dedupe = bfd_preceding[~bfd_preceding.index.duplicated(keep='first')]

In [52]:
bfd_preceding_dedupe.shape

(321564, 50)

In [97]:
len(bfd_preceding_dedupe['spn'].unique())

274

In [159]:
test = bfd_preceding_dedupe[0:100 ]
test['spn_test'] = 1
test['spn_dupe'] = test['spn']
test

/tmp/ipykernel_4264/115509719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['spn_test'] = 1
/tmp/ipykernel_4264/115509719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['spn_dupe'] = test['spn']


,,Unnamed: 0,RecordID,ESS_Id,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,fully_derated,partially_derated,first_derate_date,flipper,spn_test,spn_dupe
EventTimeStamp,EquipmentID,,,,,,,,,,,,,,,,,,,,,
2015-02-20 17:51:31,1488,1759,1760,1004537,NaN,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,0,0,2016-02-12 09:46:11,1,1,523530
2015-02-21 11:40:22,1417,5,6,990431,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,...,13.602200,3276.75,NaN,6.67,0,0,2015-07-21 11:06:35,1,1,111
2015-02-21 11:41:55,1417,9,10,990446,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,...,NaN,NaN,NaN,NaN,0,0,2015-07-21 11:06:35,1,1,111
2015-02-21 11:42:19,1582,10,11,990459,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,...,64.826490,3276.75,NaN,17.69,0,0,2016-02-17 07:43:34,1,1,111
2015-02-21 11:43:40,1582,13,14,990470,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,...,NaN,NaN,NaN,NaN,0,0,2016-02-17 07:43:34,1,1,111
2015-02-21 11:43:44,1417,61,62,991105,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,...,62.205080,3276.75,NaN,9.57,0,0,2015-07-21 11:06:35,1,1,111
2015-02-21 11:45:06,1601,15,16,990495,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,...,NaN,NaN,NaN,NaN,0,0,2016-04-24 14:07:08,1,1,111
2015-02-21 11:45:14,1601,16,17,990504,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,...,36.389050,3276.75,NaN,1.16,0,0,2016-04-24 14:07:08,0,1,111
2015-02-21 11:50:50,1590,24,25,990611,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190040*082113134117*07700053*I0*BBZ*,79620774,6X1u10D1500000000,CMMNS,0,...,7.990445,3276.75,NaN,3.48,0,0,2015-09-10 06:45:08,1,1,111


In [160]:
pivoted_test= pd.pivot(test.reset_index(), index=['EventTimeStamp','EquipmentID', 'spn'], columns='spn_dupe', values = 'spn_test')

In [161]:
pivoted_test.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100 entries, (Timestamp('2015-02-20 17:51:31'), '1488', 523530) to (Timestamp('2015-02-21 16:09:45'), '1467', 111)
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   97      1 non-null      float64
 1   111     60 non-null     float64
 2   596     3 non-null      float64
 3   627     2 non-null      float64
 4   790     2 non-null      float64
 5   929     23 non-null     float64
 6   1068    5 non-null      float64
 7   2863    2 non-null      float64
 8   5246    1 non-null      float64
 9   523530  1 non-null      float64
dtypes: float64(10)
memory usage: 14.2+ KB


In [162]:
pivoted_test

,,spn_dupe,97,111,596,627,790,929,1068,2863,5246,523530
EventTimeStamp,EquipmentID,spn,,,,,,,,,,
2015-02-20 17:51:31,1488,523530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2015-02-21 11:40:22,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:41:55,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:42:19,1582,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:43:40,1582,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:43:44,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:45:06,1601,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:45:14,1601,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:50:50,1590,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
cols1 =pivoted_test.columns.to_list()

In [164]:
pivoted_test

,,spn_dupe,97,111,596,627,790,929,1068,2863,5246,523530
EventTimeStamp,EquipmentID,spn,,,,,,,,,,
2015-02-20 17:51:31,1488,523530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2015-02-21 11:40:22,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:41:55,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:42:19,1582,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:43:40,1582,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:43:44,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:45:06,1601,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:45:14,1601,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:50:50,1590,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
pivoted_test = pivoted_test.sort_values(['EquipmentID', 'EventTimeStamp'], ascending = [True, True]).reset_index()

In [166]:
pivoted_test

spn_dupe,EventTimeStamp,EquipmentID,spn,97,111,596,627,790,929,1068,2863,5246,523530
0,2015-02-21 12:10:51,1395,5246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,2015-02-21 11:40:22,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-02-21 11:41:55,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-02-21 11:43:44,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-02-21 12:33:45,1417,111,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-02-21 12:14:49,1437,1068,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
6,2015-02-21 12:14:51,1437,1068,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
7,2015-02-21 13:51:34,1437,1068,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
8,2015-02-21 13:51:36,1437,1068,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
9,2015-02-21 13:53:00,1437,2863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [167]:
pivoted_test = pivoted_test.reset_index(drop=True)
pivoted_test =pivoted_test.fillna(0)

In [168]:
pivoted_test

spn_dupe,EventTimeStamp,EquipmentID,spn,97,111,596,627,790,929,1068,2863,5246,523530
0,2015-02-21 12:10:51,1395,5246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2015-02-21 11:40:22,1417,111,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-02-21 11:41:55,1417,111,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-02-21 11:43:44,1417,111,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-02-21 12:33:45,1417,111,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2015-02-21 12:14:49,1437,1068,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,2015-02-21 12:14:51,1437,1068,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,2015-02-21 13:51:34,1437,1068,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,2015-02-21 13:51:36,1437,1068,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,2015-02-21 13:53:00,1437,2863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [169]:
pivoted_test =pivoted_test.set_index('EventTimeStamp')

In [170]:
pivoted_test.columns

Index(['EquipmentID', 'spn', 97, 111, 596, 627, 790, 929, 1068, 2863, 5246,
       523530],
      dtype='object', name='spn_dupe')

In [175]:
df = pivoted_test.groupby('EquipmentID')[cols1].rolling(window = '10T').sum()
df

spn_dupe                          97  111  596  627  790  929  1068  2863  \
EquipmentID EventTimeStamp                                                  
1395        2015-02-21 12:10:51  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   
1417        2015-02-21 11:40:22  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 11:41:55  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 11:43:44  0.0  3.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:33:45  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
1437        2015-02-21 12:14:49  0.0  0.0  0.0  0.0  0.0  0.0   1.0   0.0   
            2015-02-21 12:14:51  0.0  0.0  0.0  0.0  0.0  0.0   2.0   0.0   
            2015-02-21 13:51:34  0.0  0.0  0.0  0.0  0.0  0.0   1.0   0.0   
            2015-02-21 13:51:36  0.0  0.0  0.0  0.0  0.0  0.0   2.0   0.0   
            2015-02-21 13:53:00  0.0  0.0  0.0  0.0  0.0  0.0   2.0   1.0   
            2015-02-21 14:02:01  0.0  0.0  0.0  0.0  0.0  0.0   1.0   1.0   
            2015-02-21 14:03:02  0.0  0.0  0.0  0.0  0.0  0.0   1.0   1.0   
1467        2015-02-21 16:00:08  1.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 16:09:45  1.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
1488        2015-02-20 17:51:31  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   
1566        2015-02-21 15:30:53  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
1582        2015-02-21 11:42:19  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 11:43:40  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:23:27  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:27:17  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:35:01  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:37:42  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
1585        2015-02-21 11:53:02  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:01:10  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:04:50  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:10:34  0.0  3.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:47:36  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
1586        2015-02-21 12:03:15  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:13:18  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:37:29  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
1590        2015-02-21 11:50:50  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:11:09  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:26:40  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:02:22  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:11:55  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:13:48  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:39:39  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:55:19  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 14:32:27  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 14:39:42  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 15:04:55  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
1595        2015-02-21 12:52:35  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:58:22  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:57:13  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:58:34  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 16:00:18  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 16:02:23  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
1600        2015-02-21 12:19:44  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 12:21:08  0.0  2.0  0.0  0.0  0.0  0.0   0.0   0.0   
            2015-02-21 13:01:31  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0

In [155]:
df.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100 entries, ('1395', Timestamp('2015-02-21 12:10:51')) to ('1768', Timestamp('2015-02-21 13:37:19'))
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   97      100 non-null    float64
 1   111     100 non-null    float64
 2   596     100 non-null    float64
 3   627     100 non-null    float64
 4   790     100 non-null    float64
 5   929     100 non-null    float64
 6   1068    100 non-null    float64
 7   2863    100 non-null    float64
 8   5246    100 non-null    float64
 9   523530  100 non-null    float64
dtypes: float64(10)
memory usage: 13.1+ KB


In [156]:
pd.set_option("display.max_rows",100)

In [157]:
df

spn                               97   111  596  627  790   929  1068  2863  \
EquipmentID EventTimeStamp                                                    
1395        2015-02-21 12:10:51  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   
1417        2015-02-21 11:40:22  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 11:41:55  0.0   2.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 11:43:44  0.0   3.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:33:45  0.0   4.0  0.0  0.0  0.0   0.0   0.0   0.0   
1437        2015-02-21 12:14:49  0.0   0.0  0.0  0.0  0.0   0.0   1.0   0.0   
            2015-02-21 12:14:51  0.0   0.0  0.0  0.0  0.0   0.0   2.0   0.0   
            2015-02-21 13:51:34  0.0   0.0  0.0  0.0  0.0   0.0   3.0   0.0   
            2015-02-21 13:51:36  0.0   0.0  0.0  0.0  0.0   0.0   4.0   0.0   
            2015-02-21 13:53:00  0.0   0.0  0.0  0.0  0.0   0.0   4.0   1.0   
            2015-02-21 14:02:01  0.0   0.0  0.0  0.0  0.0   0.0   5.0   1.0   
            2015-02-21 14:03:02  0.0   0.0  0.0  0.0  0.0   0.0   5.0   2.0   
1467        2015-02-21 16:00:08  1.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 16:09:45  1.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
1488        2015-02-20 17:51:31  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0   
1566        2015-02-21 15:30:53  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
1582        2015-02-21 11:42:19  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 11:43:40  0.0   2.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:23:27  0.0   3.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:27:17  0.0   4.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:35:01  0.0   5.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:37:42  0.0   6.0  0.0  0.0  0.0   0.0   0.0   0.0   
1585        2015-02-21 11:53:02  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:01:10  0.0   2.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:04:50  0.0   3.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:10:34  0.0   4.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:47:36  0.0   5.0  0.0  0.0  0.0   0.0   0.0   0.0   
1586        2015-02-21 12:03:15  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:13:18  0.0   2.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:37:29  0.0   3.0  0.0  0.0  0.0   0.0   0.0   0.0   
1590        2015-02-21 11:50:50  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:11:09  0.0   2.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:26:40  0.0   3.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:02:22  0.0   4.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:11:55  0.0   5.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:13:48  0.0   6.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:39:39  0.0   7.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:55:19  0.0   8.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 14:32:27  0.0   9.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 14:39:42  0.0  10.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 15:04:55  0.0  11.0  0.0  0.0  0.0   0.0   0.0   0.0   
1595        2015-02-21 12:52:35  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:58:22  0.0   2.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:57:13  0.0   3.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 13:58:34  0.0   4.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 16:00:18  0.0   5.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 16:02:23  0.0   6.0  0.0  0.0  0.0   0.0   0.0   0.0   
1600        2015-02-21 12:19:44  0.0   1.0  0.0  0.0  0.0   0.0   0.0   0.0   
            2015-02-21 12:21:08  0.0   2.0  0.0  0

In [54]:
pivoted_spn = pd.pivot(bfd_preceding_dedupe.reset_index(), index=['EventTimeStamp','EquipmentID'], columns='spn', values ='spn')

In [55]:
pivoted_spn

,spn,0,16,27,37,38,51,70,75,77,81,...,520203,520273,520302,520953,523530,523531,524033,524037,524071,524287
EventTimeStamp,EquipmentID,,,,,,,,,,,,,,,,,,,,,
2015-02-20 17:51:31,1488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,523530.0,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:40:22,1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:41:55,1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:42:19,1582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:43:40,1582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-06 11:48:52,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-06 12:00:41,1853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-06 12:11:54,1853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pivoted_test.groupby('EquipmentID')[cols1].rolling(window = '10T').sum()


In [181]:
test1 = pivoted_spn.iloc[0:10, 20:29]
test1

,spn,102,103,105,107,108,110,111,157,158
EventTimeStamp,EquipmentID,,,,,,,,,
2015-02-20 17:51:31,1488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-21 11:40:22,1417,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:41:55,1417,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:42:19,1582,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:43:40,1582,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:43:44,1417,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:45:06,1601,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:45:14,1601,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN
2015-02-21 11:50:50,1590,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN


In [185]:
test1.reset_index().set_index('EventTimeStamp').groupby('EquipmentID')[[111,108]].rolling(window ='10T').mean()

spn                                111  108
EquipmentID EventTimeStamp                 
1417        2015-02-21 11:40:22  111.0  NaN
            2015-02-21 11:41:55  111.0  NaN
            2015-02-21 11:43:44  111.0  NaN
1488        2015-02-20 17:51:31    NaN  NaN
1582        2015-02-21 11:42:19  111.0  NaN
            2015-02-21 11:43:40  111.0  NaN
1590        2015-02-21 11:50:50  111.0  NaN
1601        2015-02-21 11:45:06  111.0  NaN
            2015-02-21 11:45:14  111.0  NaN
1605        2015-02-21 11:52:46    NaN  NaN

In [67]:
er_c = pd.get_dummies(test)
er_c

,spn,27,37,38,51
EventTimeStamp,EquipmentID,,,,
2015-02-20 17:51:31,1488,NaN,NaN,NaN,NaN
2015-02-21 11:40:22,1417,NaN,NaN,NaN,NaN
2015-02-21 11:41:55,1417,NaN,NaN,NaN,NaN
2015-02-21 11:42:19,1582,NaN,NaN,NaN,NaN
2015-02-21 11:43:40,1582,NaN,NaN,NaN,NaN
2015-02-21 11:43:44,1417,NaN,NaN,NaN,NaN
2015-02-21 11:45:06,1601,NaN,NaN,NaN,NaN
2015-02-21 11:45:14,1601,NaN,NaN,NaN,NaN
2015-02-21 11:50:50,1590,NaN,NaN,NaN,NaN


In [69]:
er_c.groupby(['EventTimeStamp','EquipmentID']).sum().reset_index()

spn,EventTimeStamp,EquipmentID,27,37,38,51
0,2015-02-20 17:51:31,1488,0.0,0.0,0.0,0.0
1,2015-02-21 11:40:22,1417,0.0,0.0,0.0,0.0
2,2015-02-21 11:41:55,1417,0.0,0.0,0.0,0.0
3,2015-02-21 11:42:19,1582,0.0,0.0,0.0,0.0
4,2015-02-21 11:43:40,1582,0.0,0.0,0.0,0.0
5,2015-02-21 11:43:44,1417,0.0,0.0,0.0,0.0
6,2015-02-21 11:45:06,1601,0.0,0.0,0.0,0.0
7,2015-02-21 11:45:14,1601,0.0,0.0,0.0,0.0
8,2015-02-21 11:50:50,1590,0.0,0.0,0.0,0.0
9,2015-02-21 11:52:46,1605,0.0,0.0,0.0,0.0


In [70]:
dm=[27, 37, 38, 51]

In [73]:
er_c.reset_index().groupby('EquipmentID').rolling(window = 2, on='EventTimeStamp')[dm].sum()

spn                              27  37  38  51
EquipmentID EventTimeStamp                     
1417        2015-02-21 11:40:22 NaN NaN NaN NaN
            2015-02-21 11:41:55 NaN NaN NaN NaN
            2015-02-21 11:43:44 NaN NaN NaN NaN
1488        2015-02-20 17:51:31 NaN NaN NaN NaN
1582        2015-02-21 11:42:19 NaN NaN NaN NaN
            2015-02-21 11:43:40 NaN NaN NaN NaN
1590        2015-02-21 11:50:50 NaN NaN NaN NaN
1601        2015-02-21 11:45:06 NaN NaN NaN NaN
            2015-02-21 11:45:14 NaN NaN NaN NaN
1605        2015-02-21 11:52:46 NaN NaN NaN NaN

In [82]:
pivoted_cols = pivoted_spn.columns.to_list()

In [78]:
spn_count = pd.get_dummies(pivoted_spn)


In [86]:
 spn_cols = ['EquipmentID', 'EventTimeStamp',
 0,
 16,
 27,
 37,
 38,
 51,
 70,
 75,
 77,
 81,
 84,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 105,
 107,
 108,
 110,
 111,
 157,
 158,
 167,
 168,
 171,
 173,
 175,
 184,
 188,
 190,
 228,
 235,
 236,
 237,
 245,
 247,
 248,
 251,
 252,
 256,
 411,
 412,
 442,
 444,
 512,
 558,
 563,
 576,
 578,
 596,
 611,
 612,
 614,
 627,
 628,
 629,
 630,
 633,
 639,
 641,
 647,
 651,
 652,
 653,
 654,
 655,
 656,
 723,
 729,
 788,
 789,
 790,
 791,
 792,
 793,
 794,
 795,
 796,
 797,
 798,
 799,
 800,
 802,
 803,
 805,
 806,
 807,
 810,
 815,
 829,
 830,
 862,
 886,
 917,
 929,
 933,
 934,
 935,
 938,
 939,
 940,
 941,
 976,
 1024,
 1028,
 1043,
 1045,
 1056,
 1059,
 1067,
 1068,
 1072,
 1073,
 1075,
 1081,
 1127,
 1172,
 1176,
 1209,
 1231,
 1239,
 1321,
 1322,
 1323,
 1324,
 1325,
 1326,
 1328,
 1347,
 1349,
 1481,
 1482,
 1483,
 1487,
 1569,
 1659,
 1675,
 1761,
 1807,
 1808,
 1809,
 1815,
 2017,
 2029,
 2623,
 2629,
 2659,
 2791,
 2863,
 2866,
 3031,
 3058,
 3060,
 3216,
 3218,
 3226,
 3228,
 3241,
 3242,
 3245,
 3246,
 3249,
 3251,
 3253,
 3360,
 3361,
 3362,
 3363,
 3364,
 3464,
 3480,
 3482,
 3490,
 3509,
 3510,
 3511,
 3513,
 3556,
 3583,
 3597,
 3610,
 3697,
 3698,
 3703,
 3720,
 3936,
 4094,
 4095,
 4096,
 4219,
 4220,
 4276,
 4331,
 4334,
 4339,
 4340,
 4342,
 4344,
 4346,
 4354,
 4356,
 4360,
 4363,
 4364,
 4375,
 4376,
 4752,
 4765,
 4792,
 4794,
 4796,
 4811,
 4813,
 5019,
 5024,
 5031,
 5052,
 5245,
 5246,
 5298,
 5319,
 5394,
 5397,
 5443,
 5485,
 5491,
 5585,
 5742,
 5743,
 5745,
 5746,
 5835,
 5848,
 5851,
 5862,
 5909,
 5939,
 6773,
 6780,
 6802,
 7827,
 7847,
 7854,
 13600,
 17096,
 17590,
 25780,
 29902,
 35527,
 36017,
 47284,
 50353,
 51923,
 65535,
 78132,
 139296,
 520201,
 520203,
 520273,
 520302,
 520953,
 523530,
 523531,
 524033,
 524037,
 524071,
 524287]

In [87]:
spn_count.reset_index(inplace=True)

In [88]:
spn_count

spn,EventTimeStamp,EquipmentID,0,16,27,37,38,51,70,75,...,520203,520273,520302,520953,523530,523531,524033,524037,524071,524287
0,2015-02-20 17:51:31,1488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,523530.0,NaN,NaN,NaN,NaN,NaN
1,2015-02-21 11:40:22,1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-02-21 11:41:55,1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-02-21 11:42:19,1582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-02-21 11:43:40,1582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321559,2020-03-06 11:48:52,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321560,2020-03-06 12:00:41,1853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321561,2020-03-06 12:11:54,1853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321562,2020-03-06 12:20:36,1853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
spn_count.columns = ['EventTimeStamp',
 'EquipmentID',
 0,
 16,
 27,
 37,
 38,
 51,
 70,
 75,
 77,
 81,
 84,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 105,
 107,
 108,
 110,
 111,
 157,
 158,
 167,
 168,
 171,
 173,
 175,
 184,
 188,
 190,
 228,
 235,
 236,
 237,
 245,
 247,
 248,
 251,
 252,
 256,
 411,
 412,
 442,
 444,
 512,
 558,
 563,
 576,
 578,
 596,
 611,
 612,
 614,
 627,
 628,
 629,
 630,
 633,
 639,
 641,
 647,
 651,
 652,
 653,
 654,
 655,
 656,
 723,
 729,
 788,
 789,
 790,
 791,
 792,
 793,
 794,
 795,
 796,
 797,
 798,
 799,
 800,
 802,
 803,
 805,
 806,
 807,
 810,
 815,
 829,
 830,
 862,
 886,
 917,
 929,
 933,
 934,
 935,
 938,
 939,
 940,
 941,
 976,
 1024,
 1028,
 1043,
 1045,
 1056,
 1059,
 1067,
 1068,
 1072,
 1073,
 1075,
 1081,
 1127,
 1172,
 1176,
 1209,
 1231,
 1239,
 1321,
 1322,
 1323,
 1324,
 1325,
 1326,
 1328,
 1347,
 1349,
 1481,
 1482,
 1483,
 1487,
 1569,
 1659,
 1675,
 1761,
 1807,
 1808,
 1809,
 1815,
 2017,
 2029,
 2623,
 2629,
 2659,
 2791,
 2863,
 2866,
 3031,
 3058,
 3060,
 3216,
 3218,
 3226,
 3228,
 3241,
 3242,
 3245,
 3246,
 3249,
 3251,
 3253,
 3360,
 3361,
 3362,
 3363,
 3364,
 3464,
 3480,
 3482,
 3490,
 3509,
 3510,
 3511,
 3513,
 3556,
 3583,
 3597,
 3610,
 3697,
 3698,
 3703,
 3720,
 3936,
 4094,
 4095,
 4096,
 4219,
 4220,
 4276,
 4331,
 4334,
 4339,
 4340,
 4342,
 4344,
 4346,
 4354,
 4356,
 4360,
 4363,
 4364,
 4375,
 4376,
 4752,
 4765,
 4792,
 4794,
 4796,
 4811,
 4813,
 5019,
 5024,
 5031,
 5052,
 5245,
 5246,
 5298,
 5319,
 5394,
 5397,
 5443,
 5485,
 5491,
 5585,
 5742,
 5743,
 5745,
 5746,
 5835,
 5848,
 5851,
 5862,
 5909,
 5939,
 6773,
 6780,
 6802,
 7827,
 7847,
 7854,
 13600,
 17096,
 17590,
 25780,
 29902,
 35527,
 36017,
 47284,
 50353,
 51923,
 65535,
 78132,
 139296,
 520201,
 520203,
 520273,
 520302,
 520953,
 523530,
 523531,
 524033,
 524037,
 524071,
 524287]


In [90]:
spn_count=spn_count.groupby(['EventTimeStamp', 'EquipmentID']).sum().reset_index()
spn_count

,EventTimeStamp,EquipmentID,0,16,27,37,38,51,70,75,...,520203,520273,520302,520953,523530,523531,524033,524037,524071,524287
0,2015-02-20 17:51:31,1488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,523530.0,0.0,0.0,0.0,0.0,0.0
1,2015-02-21 11:40:22,1417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-02-21 11:41:55,1417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-02-21 11:42:19,1582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-02-21 11:43:40,1582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321559,2020-03-06 11:48:52,1970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321560,2020-03-06 12:00:41,1853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321561,2020-03-06 12:11:54,1853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321562,2020-03-06 12:20:36,1853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
spn_count.describe()

,0,16,27,37,38,51,70,75,77,81,...,520203,520273,520302,520953,523530,523531,524033,524037,524071,524287
count,321564.0,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,...,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000,321564.000000
mean,0.0,0.000050,0.005290,0.009435,0.000118,0.003806,0.309114,0.001166,0.000718,0.043326,...,215.157788,14.561509,14.562320,21.060781,30.933407,187.228872,40.740957,13.037206,3.259513,96.195261
std,0.0,0.028215,0.377884,0.590772,0.067012,0.440582,4.641391,0.295740,0.235189,1.872837,...,10577.323950,2752.411918,2752.565337,3312.290391,4024.134678,9898.755915,4620.390928,2613.776849,1306.985410,7101.044165
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.0,16.000000,27.000000,37.000000,38.000000,51.000000,70.000000,75.000000,77.000000,81.000000,...,520203.000000,520273.000000,520302.000000,520953.000000,523530.000000,523531.000000,524033.000000,524037.000000,524071.000000,524287.000000


In [95]:
temp = []


for col in spn_count.columns[2:]:

    temp.append(pd.pivot_table(spn_count,

                               index='EventTimeStamp',

                               columns='EquipmentID',

                               values=col).rolling(window='7D').sum().resample('24H',

                                                                             closed='left',

                                                                             label='right').first().unstack())
error_count = pd.concat(temp, axis=1)

error_count.columns = spn_count.columns[2:]

error_count.reset_index(inplace=True)

error_count = error_count.dropna()

error_count.describe()

,0,16,27,37,38,51,70,75,77,81,...,520203,520273,520302,520953,523530,523531,524033,524037,524071,524287
count,107621.0,107621.000000,107621.000000,107621.000000,107621.000000,107621.000000,107621.000000,107621.000000,107621.000000,107621.000000,...,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05,1.076210e+05
mean,0.0,0.001041,0.111642,0.201122,0.002472,0.078665,6.449020,0.024391,0.015025,0.921233,...,4.601641e+03,3.190643e+02,3.045783e+02,4.647001e+02,6.761754e+02,4.008414e+03,8.618563e+02,2.726798e+02,6.817437e+01,1.997358e+03
std,0.0,0.129035,3.197311,4.712539,0.306459,4.765719,99.227295,2.180762,1.348262,10.881080,...,8.748199e+04,1.537288e+04,3.441933e+04,2.344202e+04,3.676791e+04,7.186159e+04,3.822778e+04,1.581117e+04,8.452954e+03,5.105237e+04
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,0.0,16.000000,216.000000,296.000000,38.000000,510.000000,3360.000000,225.000000,154.000000,324.000000,...,4.161624e+06,1.040546e+06,4.682718e+06,2.083812e+06,4.711770e+06,3.141186e+06,4.192264e+06,1.048074e+06,1.048142e+06,4.194296e+06


In [96]:
error_count.head()

,EquipmentID,EventTimeStamp,0,16,27,37,38,51,70,75,...,520203,520273,520302,520953,523530,523531,524033,524037,524071,524287
5,1329,2015-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1329,2015-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1329,2015-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1329,2015-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1329,2015-03-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dfx_seven_day.shape

In [ ]:
len(dfx_seven_day['spn'].unique())

In [ ]:
dfx_seven_day[dfx_seven_day['EquipmentID']=='1872']

In [ ]:
dfx_seven_day[dfx_seven_day['EquipmentID']=='1872'][['spn']]

In [ ]:
dfx_seven_day.columns

In [ ]:
# calculating cross sectional stats for time delta between spn codes per a truck 
# filtering events that occurred less than 30 times so that stdev is statitically significant
result = []

dfx_seven_day_copy = dfx_seven_day.copy()
dfx_seven_day_copy.sort_values(['EquipmentID', 'EventTimeStamp'], ascending = [True, True], inplace =True)
for spn in dfx_seven_day_copy['spn'].unique():
    temp = dfx_seven_day_copy.loc[dfx_seven_day_copy['spn'] == spn,"EventTimeStamp"].diff()
    if len(temp) ==0:
        continue 
    temp = temp.apply(lambda x: x/np.timedelta64(1, 'D')).fillna(0).astype('int64')
    filtered_temp = temp[temp > 0]
    spn_dev = filtered_temp.std()
    spn_mean=filtered_temp.mean()
    result.append((spn, len(filtered_temp),spn_mean, spn_dev))
spn_stats = pd.DataFrame(result, columns = ['spn','sample_size', 'spn_mean', 'spn_dev'])
spn_stats.loc[spn_stats['sample_size'] > 30,:].sort_values(['spn_dev'], ascending=True)

In [ ]:
#temp = dfx_seven_day_copy.loc[(dfx_seven_day_copy['spn'] == spn) & (dfx_seven_day_copy['flipper']!=1),"EventTimeStamp"].diff()
    

In [ ]:
spns = [1569, 3364, 111, 3362, 3216, 3226, 1761, 96, 3031, 5394, 829]

In [ ]:
sensors = dfx_seven_day.loc[dfx_seven_day['spn'].isin(spns)][['EventTimeStamp','AcceleratorPedal', 'BarometricPressure', 'CruiseControlActive',
       'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature',
       'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm',
       'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature',
       'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure']]

In [ ]:
sensors.set_index('EventTimeStamp')

In [ ]:
sensors = sensors.fillna(0)

In [ ]:
cols =['BarometricPressure', 'EngineCoolantTemperature', 'FuelTemperature','SwitchedBatteryVoltage', 'Throttle']

In [ ]:
plt.rcParams['figure.figsize'] = (20,20)
plt.style.use('fivethirtyeight')
sensors[['BarometricPressure', 'EngineCoolantTemperature', 'FuelTemperature','SwitchedBatteryVoltage', 'Throttle']].plot(subplots= True,layout=(7,1),sharex=False, sharey=False, colormap='hsv',linewidth = 2);
plt.tight_layout();

In [ ]:
corr = sensors.corr()
top_features = corr.index
plt.figure(figsize=(20,20))
sns.heatmap(sensors[top_features].corr(), annot =True)

In [ ]:
threshold = 0.65

In [ ]:
def correlation(sensors, threshold):
    col_corr = set() # Set of all the names of correlated columns
    corr_matrix = sensors.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
              if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i] # getting the nhesame of column
                    col_corr.add(colname)
    return col_corr  

In [ ]:
correlation(sensors, threshold)

In [ ]:
temp = []


for col in spn_count.columns[2:6]:

    temp.append(pd.pivot_table(spn_count,

                               index='EventTimeStamp',

                               columns='EquipmentID',

                               values=col).rolling(window='1D').sum().resample('3H',

                                                                             closed='left',

                                                                             label='right').first().unstack())
error_count = pd.concat(temp, axis=1)

error_count.columns = spn_count.columns[2:6]

error_count.reset_index(inplace=True)

error_count = error_count.dropna()

error_count.describe()